<a href="https://colab.research.google.com/github/DimaFrank/Association_Rule_Learning/blob/test/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=0c7c545eecfb10f5b7e24fc3f34ce8421849cef405aaf7531d778b21bb6c95ec
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import array, col, concat_ws, udf, array_remove, size, window, to_timestamp, date_format, concat, lit, collect_list, desc, sort_array, array_contains, array_intersect
from pyspark.sql import SparkSession 
import itertools

Resource:

https://www.softwaretestinghelp.com/apriori-algorithm/

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("user_ct_test").getOrCreate()
sc = spark.sparkContext

In [ ]:
# TABLE-1
rdd = spark.sparkContext.parallelize(
    [Row("T1", ['I1', 'I2', 'I3']),
     Row("T2", ['I4', 'I3', 'I2']),
     Row("T3", ['I4', 'I5']), 
     Row("T4", ['I1', 'I2', 'I4']),
     Row("T5", ['I1', 'I2', 'I3', 'I5']),
     Row("T6", ['I1', 'I2', 'I3', 'I4']),   
     ]
)
schema = StructType([
    StructField("Tid", StringType(), True),
    StructField("Basket", ArrayType(StringType(), True), True)
])
df = spark.createDataFrame(rdd, schema)
df = df.withColumn('size',size(col('Basket')))
df.show(10)

+---+----------------+----+
|Tid|          Basket|size|
+---+----------------+----+
| T1|    [I1, I2, I3]|   3|
| T2|    [I4, I3, I2]|   3|
| T3|        [I4, I5]|   2|
| T4|    [I1, I2, I4]|   3|
| T5|[I1, I2, I3, I5]|   4|
| T6|[I1, I2, I3, I4]|   4|
+---+----------------+----+



In [ ]:
df_test = df.select("Basket",sort_array("Basket",asc=True).alias('array_sorted'))
df_test.show()

+----------------+----------------+
|          Basket|    array_sorted|
+----------------+----------------+
|    [I1, I2, I3]|    [I1, I2, I3]|
|    [I4, I3, I2]|    [I2, I3, I4]|
|        [I4, I5]|        [I4, I5]|
|    [I1, I2, I4]|    [I1, I2, I4]|
|[I1, I2, I3, I5]|[I1, I2, I3, I5]|
|[I1, I2, I3, I4]|[I1, I2, I3, I4]|
+----------------+----------------+



In [ ]:
min_sup=3

In [ ]:
# TABLE-2
import pyspark.sql.functions as F

F1 = (df_test
           .withColumn("explode", F.explode("array_sorted"))
           .groupBy("explode")
           .count()
           .orderBy(F.desc("count")))
F1.show()

+-------+-----+
|explode|count|
+-------+-----+
|     I2|    5|
|     I4|    4|
|     I3|    4|
|     I1|    4|
|     I5|    2|
+-------+-----+



In [ ]:
# TABLE-3
F2 = F1.filter(col('count')>=min_sup)
F2.show()

+-------+-----+
|explode|count|
+-------+-----+
|     I2|    5|
|     I4|    4|
|     I1|    4|
|     I3|    4|
+-------+-----+



In [ ]:
# All possible combinations

lst = [str(i.explode) for i in F2.select("explode").collect()]
print(lst)
print('All Combinations: \n')

def create_possible_combinations(all_items, k):
    # This functions gets a list of items and k, and returns all possible k-item combinations.
    # all_items --> <list>
    # k         --> <int>
    res_tmp = set([])
    for subset in itertools.combinations(all_items, k):       
       res_tmp.add((subset))

    res_lst = list(res_tmp)
    result = [list(res_lst[i]) for i in range(len(res_lst))]

    return result


create_possible_combinations(lst,2)

['I2', 'I4', 'I3', 'I1']
All Combinations: 



[['I2', 'I1'],
 ['I2', 'I4'],
 ['I3', 'I1'],
 ['I4', 'I3'],
 ['I2', 'I3'],
 ['I4', 'I1']]

In [ ]:
# Creating an empty DataFrame
emp_RDD = spark.sparkContext.emptyRDD()
columns1 = StructType([StructField('Item', ArrayType(StringType()), False),
                       StructField('count', LongType(), False)])
first_df = spark.createDataFrame(data=emp_RDD,
                                         schema=columns1)


for row in create_possible_combinations(lst,2):
    # print(row)
    res=df_test.withColumn("NewColumn", F.array([F.lit(x) for x in row]))
    res= res.select('array_sorted', size(array_intersect(res.array_sorted, res.NewColumn)).alias('Intersect'))
    res = res.filter(col('Intersect')>=2).count()
    # print(row, 'res=', res)
    columns=['Item','count']
    newRow = spark.createDataFrame([(row, res)], columns)
    first_df = first_df.union(newRow).filter(col('count')>=min_sup)
    

first_df.show()  

+--------+-----+
|    Item|count|
+--------+-----+
|[I2, I1]|    4|
|[I2, I4]|    3|
|[I3, I1]|    3|
|[I2, I3]|    4|
+--------+-----+



In [ ]:
for row in create_possible_combinations(lst,3):
    # print(row)
    res=df_test.withColumn("NewColumn", F.array([F.lit(x) for x in row]))
    res= res.select('array_sorted', size(array_intersect(res.array_sorted, res.NewColumn)).alias('Intersect'))
    res = res.filter(col('Intersect')>=3).count()
    # print(row, 'res=', res)
    columns=['Item','count']
    newRow = spark.createDataFrame([(row, res)], columns)
    first_df = first_df.union(newRow).filter(col('count')>=min_sup)


first_df.show()  

+------------+-----+
|        Item|count|
+------------+-----+
|    [I2, I1]|    4|
|    [I2, I4]|    3|
|    [I3, I1]|    3|
|    [I2, I3]|    4|
|[I2, I3, I1]|    3|
+------------+-----+



In [ ]:
df_test = df_test.groupBy('array_sorted').count().orderBy(col("count").desc())
df_test.show(10)

+----------------+-----+
|    array_sorted|count|
+----------------+-----+
|        [I4, I5]|    1|
|    [I1, I2, I3]|    1|
|    [I2, I3, I4]|    1|
|[I1, I2, I3, I4]|    1|
|[I1, I2, I3, I5]|    1|
|    [I1, I2, I4]|    1|
+----------------+-----+



In [ ]:
df_test.printSchema()

root
 |-- array_sorted: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- count: long (nullable = false)



In [ ]:
# Get the fist two elements 
split_row = udf(lambda row: row[:2])

def array_size(x):
  if size(x)>0:
    return 1
  return 0

array_size_UDF = udf(lambda z: array_size(z))

In [ ]:
df_test.show()

+----------------+-----+
|    array_sorted|count|
+----------------+-----+
|        [I4, I5]|    1|
|    [I1, I2, I3]|    1|
|    [I2, I3, I4]|    1|
|[I1, I2, I3, I4]|    1|
|[I1, I2, I3, I5]|    1|
|    [I1, I2, I4]|    1|
+----------------+-----+



In [ ]:
def create_item_sets(dataset, items_col, display=False, k=0):
  #This function creates sets of n items from a given data and column. Where: n = 1,2....N
  array = dataset.withColumn('size',size(col(items_col)))
  collection = []
  max_items = array.agg({"size": "max"}).collect()[0][0]
  min_items = array.agg({"size": "min"}).collect()[0][0]
  for n in range(min_items, max_items+1):
    tmp = dataset.filter(size(col(items_col))==n).select(items_col)
    collection.append(tmp)
    if display==True:
      tmp.show()

  return collection[k]


create_item_sets(df_test,'array_sorted', display=True)

+------------+
|array_sorted|
+------------+
|    [I4, I5]|
+------------+

+------------+
|array_sorted|
+------------+
|[I1, I2, I3]|
|[I2, I3, I4]|
|[I1, I2, I4]|
+------------+

+----------------+
|    array_sorted|
+----------------+
|[I1, I2, I3, I4]|
|[I1, I2, I3, I5]|
+----------------+



DataFrame[array_sorted: array<string>]

In [ ]:
for row in create_item_sets(df_test,'array_sorted')[1].rdd.toLocalIterator():
    row = row.array_sorted
    print(row)
    res=df_test.withColumn("NewColumn", F.array([F.lit(x) for x in row]))
    # df_test.select("array_sorted",array_intersect('array_sorted',StringType(row))).show()
    res= res.select('array_sorted', size(array_intersect(res.array_sorted, res.NewColumn)).alias('Intersect'))
    res = res.filter(col('Intersect')>=3).count()
    print(res)


['I1', 'I2', 'I3']
3
['I2', 'I3', 'I4']
2
['I1', 'I2', 'I4']
2


In [ ]:
# All possible combinations
import itertools

stuff = ['I1', 'I2', 'I3', 'I4', 'I5']
for L in range(len(stuff) + 1):
    for subset in itertools.combinations(stuff, L):       
        print(list(subset))

[]
['I1']
['I2']
['I3']
['I4']
['I5']
['I1', 'I2']
['I1', 'I3']
['I1', 'I4']
['I1', 'I5']
['I2', 'I3']
['I2', 'I4']
['I2', 'I5']
['I3', 'I4']
['I3', 'I5']
['I4', 'I5']
